In [234]:
import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\wnm\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [235]:
df = pd.read_csv('base.csv').dropna()
df.sample(3)

,Message ID,Subject,Message,Spam/Ham,Date
10659,10659,"all graphics software available , cheap oem ve...","good morning ,\nwe we offer latest oem package...",spam,2005-07-13
6763,6763,risk bucketing for p / l,"ken and greg ,\nwhat we have been doing is abs...",ham,2000-08-10
25072,25072,herald community herald - article on pc ' s,"microsoft , adobe , macromedia and many more\n...",spam,2005-05-25


In [236]:
df['Message'] = df['Subject'] +' '+ df['Message']

In [237]:
df['Spam/Ham'] = np.where(df['Spam/Ham'] == 'spam', 1, 0)

In [238]:
df.sample(3)

,Message ID,Subject,Message,Spam/Ham,Date
19693,19693,re : beautiful mohter to date with you,re : beautiful mohter to date with you ellolll...,1,2004-08-29
9511,9511,valuation methodology,valuation methodology we ' ve had a request fr...,0,2001-05-11
361,361,half day of vac on 2 / 28,half day of vac on 2 / 28 12 . 20 dentist appt...,0,2000-02-25


In [239]:
df = df.drop(['Subject','Message ID', 'Date'], axis=1)

In [240]:
def clean_text(text: str):
    text = text.lower()
    # Удаление спец символов
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    # Удаление множественных пробелов
    text = re.sub(r'\s+', ' ', text)
    text = text.replace('\n', '')
    # Фильтрация стоп слов
    stop_words = set(stopwords.words('english'))
    filtered_sentence = [word for word in text.split(" ") if not word in stop_words]
    return ' '.join(filtered_sentence)

In [241]:
clean_text(df['Message'][1])

'vastar resources inc gary production high island larger block 1 2 commenced saturday 2 00 p 6 500 gross carlos expects 9 500 10 000 gross tomorrow vastar owns 68 gross production george x 3 6992 forwarded george weissman hou ect 12 13 99 10 16 daren j farmer 12 10 99 10 38 carlos j rodriguez hou ect ect cc george weissman hou ect ect melissa graves hou ect ect subject vastar resources inc carlos please call linda get everything set going estimate 4 500 coming tomorrow 2 000 increase following day based conversations bill fischer bmar forwarded daren j farmer hou ect 12 10 99 10 34 enron north america corp george weissman 12 10 99 10 00 daren j farmer hou ect ect cc gary bryan hou ect ect melissa graves hou ect ect subject vastar resources inc darren attached appears nomination vastar resources inc high island larger block 1 2 previously erroneously referred 1 well vastar expects well commence production sometime tomorrow told linda harris get telephone number gas control provide notif

In [242]:
df['Message'] = df['Message'].apply(clean_text)

In [243]:
df.sample(3)

,Message,Spam/Ham
30757,fw w p n allied retail trade co ltd message al...,1
24031,finally remembered sally tiki person according...,0
20373,pharmaceauticals filled licensed rx us two zen...,1


In [244]:
counts = df['Spam/Ham'].value_counts()
spam_perc = round((counts[1] / df.shape[0]) * 100, 2)
print(f'Spam letters: {spam_perc}%, Normal letters {abs(1 - spam_perc)}')

Spam letters: 50.18%, Normal letters 49.18


In [245]:
df = df.sample(frac=1)
valid_size = 1000
x_train = df[0:-valid_size]
x_valid = df[-valid_size:df.shape[0]]

In [246]:
spam_prob: dict = {}
non_spam_prob: dict = {}
def calc_word_probs(dtfr):
    spam: pd.DataFrame = dtfr[dtfr['Spam/Ham'] == 1]
    non_spam : pd.DataFrame =  dtfr[dtfr['Spam/Ham'] == 0]
    spam_word_cnt = 0
    non_spam_word_cnt = 0
    for _, row in spam.iterrows():
        for word in row['Message'].split(' '):
            spam_word_cnt += 1
            if word in spam_prob:
                spam_prob[word]['cnt'] += 1
            else:
                spam_prob[word] = {'cnt': 1}

    for _, row in non_spam.iterrows():
        for word in row['Message'].split(' '):
            non_spam_word_cnt += 1
            if word in non_spam_prob:
                non_spam_prob[word]['cnt'] += 1
            else:
                non_spam_prob[word] = {'cnt': 1}

    for key in spam_prob.keys():
        spam_prob[key]["val"] = spam_prob[key]['cnt'] / spam_word_cnt
    for key in non_spam_prob.keys():
        non_spam_prob[key]["val"] = non_spam_prob[key]['cnt'] / non_spam_word_cnt
calc_word_probs(x_train)

In [251]:
def print_top_10(table: dict):
    item_list = [(word, data) for word, data in table.items()]
    sorted_spam_prob = sorted(item_list, key=lambda x: x[1]['val'], reverse=True)
    top_10_spam_words = sorted_spam_prob[:10]

    for word, data in top_10_spam_words:
        print(f"Word: '{word}' Count: {data['cnt']}, Value: {data['val']}")

In [252]:
print_top_10(non_spam_prob)

Word: 'enron' Count: 59167, Value: 0.020837126549437066
Word: 'ect' Count: 34184, Value: 0.012038743454391074
Word: 'hou' Count: 16448, Value: 0.005792571154277568
Word: '2001' Count: 15153, Value: 0.005336504784822956
Word: '1' Count: 13304, Value: 0.004685333574690464
Word: '2000' Count: 13232, Value: 0.004659976988898393
Word: 'please' Count: 12614, Value: 0.004442332960849784
Word: 'com' Count: 12187, Value: 0.0042919543201106955
Word: 'would' Count: 12087, Value: 0.00425673683984393
Word: 'subject' Count: 11732, Value: 0.004131714784896913


In [253]:
print_top_10(spam_prob)

Word: '' Count: 12675, Value: 0.005630698166591368
Word: 'com' Count: 11157, Value: 0.004956347096225633
Word: 'company' Count: 10784, Value: 0.004790646866155528
Word: '1' Count: 9647, Value: 0.004285549918193841
Word: 'e' Count: 8645, Value: 0.0038404249033674462
Word: '2' Count: 8538, Value: 0.003792891593400955
Word: 'http' Count: 8420, Value: 0.003740471681475292
Word: '3' Count: 7918, Value: 0.0035174649375203514
Word: 'email' Count: 7781, Value: 0.003456604531301573
Word: 'information' Count: 7512, Value: 0.0033371049015727306
